In [4]:
import pickle
import os
import random
import sqlite3
import random
import numpy 

In [5]:
conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()

In [6]:
import gzip
user_item = set()
with gzip.open("train_data.txt.gz",'rb') as f:
    for line in f:
        user_id,item_id,rating,text = line.split("\t")
        user_item.add(user_id+"-"+item_id)
    

In [7]:
q=Select("select user_id, business_id, review_id,  starts from Reviews ")

In [8]:
user_review_dict = {}
item_review_dict = {}

In [9]:
i=0
for row in q:
    if i % 100000 == 0:
        print i
    user = row[0]
    item = row[1]
    review_id = row[2]
    rating = row[3]
    if user+"-"+item in user_item:
        if user not in user_review_dict:
            user_review_dict[user] = []
        if item not in item_review_dict:
            item_review_dict[item] = []
        user_review_dict[user] += [(review_id,rating)]
        item_review_dict[item] += [(review_id,rating)]
    i += 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000


In [10]:
pickle.dump([user_review_dict,item_review_dict],open("user_item_rev_rating.pkl","wb"))

In [11]:
del user_item
conn.close()

In [12]:
len(list(user_review_dict.keys()))

1050896

In [41]:
len(list(item_review_dict.keys()))

156422

In [13]:
threshold=10

In [47]:
user_review_sample={}
item_review_sample={}

In [48]:
review_rating_pairs=set()
for u in user_review_dict:
    for p in user_review_dict[u]:
        review_rating_pairs.add(p)
review_rating_pairs=list(review_rating_pairs)

In [49]:
len(user_review_rating_pairs)

3789518

In [50]:
j=0
for u in user_review_dict:
    if j % 10000==0:
        print j,
    reviews=user_review_dict[u]
    if len(reviews)<threshold+1:
        reviews+=random.sample(user_review_rating_pairs,threshold+1-len(reviews)) 
    else:
        reviews=random.sample(reviews,threshold+1) 
    reviews=sorted(reviews, key=lambda x: x[1])
    user_review_sample[u]=reviews  
    j += 1

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 560000 570000 580000 590000 600000 610000 620000 630000 640000 650000 660000 670000 680000 690000 700000 710000 720000 730000 740000 750000 760000 770000 780000 790000 800000 810000 820000 830000 840000 850000 860000 870000 880000 890000 900000 910000 920000 930000 940000 950000 960000 970000 980000 990000 1000000 1010000 1020000 1030000 1040000 1050000


In [51]:
user_review_sample.items()[:2]

[(u'kiQv0q-oOVGEOPa_S_at6w',
  [(u'mJHLCMMyj8Ke-X711EM61Q', u'1'),
   (u'VkMCwedtihadjqERM1Ya-A', u'2'),
   (u'VWWrmiGEVpEICL3pYPaRug', u'4'),
   (u'zZfLAZRZntt5DTJbzIVAuA', u'4'),
   (u'zlDwnlqt54gY18gLNIiZMQ', u'4'),
   (u'Kk_7EhWFbTVi-MmQk1P7lA', u'5'),
   (u'flEXVkATYS2ApoqifUFh0A', u'5'),
   (u'K9BAo2gC1IDXJVNYHMBA1g', u'5'),
   (u'QD9d4-I47x3ci1EfLwEWwA', u'5'),
   (u'NqCXpDaAsLJgcFghxhinkg', u'5'),
   (u'_IWudMDujEtfKvFupehYAw', u'5')]),
 (u'an8MLCN3dYT8J6eSaszWlw',
  [(u'sQ992SsdFL-r05DcENoziQ', u'1'),
   (u'UYVA1Uz08I03gHacj6jPLQ', u'1'),
   (u'voSAqd3GQTAQQjwdfrLNiw', u'4'),
   (u'80FeNGPo-JWioFR2BYhXAQ', u'4'),
   (u'CVrgcmNkbIha2uWmHbu1Sg', u'4'),
   (u'w1tAMRysqBCNQrIeJjessQ', u'4'),
   (u'v9x-0eniVQ9BEnrC8TTswQ', u'5'),
   (u'puV1KWfmQF2yItxztANM-Q', u'5'),
   (u'uvkXhwvXDx8iGvwFgholSw', u'5'),
   (u'RNZDoNp0Dv-_nGXPUDbTcw', u'5'),
   (u'7oCDoL8AUFY48a19bZ32Sw', u'5')])]

In [52]:
j=0
for i in item_review_dict:
    if j % 10000==0:
        print j,
    reviews=item_review_dict[i]
    if len(reviews)<threshold+1:
        reviews+=random.sample(review_rating_pairs,threshold+1-len(reviews)) 
    else:
        reviews=random.sample(reviews,threshold+1) 
    reviews=sorted(reviews, key=lambda x: x[1])
    item_review_sample[i]=reviews  
    j += 1

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000


In [53]:
item_review_sample.items()[:2]

[(u'krBpN5vbCQrB54QvTXOzaA',
  [(u'jbznKB-lcuHDu8DMTJPpyg', u'1'),
   (u'cYf1t7QgU4XMNYe_GSTpPg', u'1'),
   (u'XZzSjF8PGrSLyRIO4JRMtQ', u'3'),
   (u'9Td3RZe43PpCCMSPPANb0A', u'4'),
   (u'rtZI0DL4XyEAkPLXM2O_xQ', u'5'),
   (u'FIeJHY_wF9cyd5ZcjG4D3g', u'5'),
   (u'UdAeD4IW5HFtaWGUitRCTg', u'5'),
   (u'5Xnt_JjEfomPdx7hWEfeVg', u'5'),
   (u'sB-ebaPWm_roDg8JVUTYEA', u'5'),
   (u'zHtaxUioRhSTIaCnBo2gdA', u'5'),
   (u'GNUWasKIPmjDttvjtEg7Gg', u'5')]),
 (u'MH0oOCJ7DKnIJWwUQcohJw',
  [(u'6h98txbP06dK2ZkfGzC-Ww', u'1'),
   (u'n72h494tzdrSV5gfWwgWyA', u'3'),
   (u'qazvNaJ3l0nnyzeXCLWtaA', u'3'),
   (u'J36igC8W0DRoEh2Ns9UAog', u'4'),
   (u'Dqxm8NnAcPPPTmveXfeDeQ', u'4'),
   (u'oUtsX2nwUCzXKsDlxpILUg', u'4'),
   (u'0ISvXKOuV4gOmQ5wPA3oAQ', u'4'),
   (u'JBz8-snIzRsNlOh7Owpbjg', u'5'),
   (u'OFer6j0KAgez2OznyWPHXg', u'5'),
   (u'xJc2jIZ--5HrbB461RrgqA', u'5'),
   (u'3bZnIXYQHvbG57pmphjBdA', u'5')])]

In [54]:
pickle.dump([user_review_sample,item_review_sample],open("user_item_rev_samples.pkl","wb"))